## Sumarry

First part on mini-project about Segmenting and Clustering Neighborhoods in Toronto. 

This notebook scraps data from wikipedia and pre-processes it

In [66]:
import pandas as pd
import numpy as np
import geocoder
#import folium


#!conda install -c anaconda beautifulsoup4
#!conda install -c conda-forge geocoder
#!conda install -c conda-forge folium=0.5.0 --yes


In [61]:
from bs4 import BeautifulSoup
import requests

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

#My_table = soup.find('table',{'class':'wikitable sortable jquery-tablesorter'})
My_table = soup.find('tbody')

postCodeList = []
boroughList = []
neighbourhoodList = []

for row in My_table.find_all('tr'):
    try:
        rowValue = row.find_all('td')
        postCodeList.append(rowValue[0].text)
        boroughList.append(rowValue[1].text)
        neighbourhoodList.append((rowValue[2].text)[:-1])
    except IndexError:pass
    

#print(neighbourhoodList)

    
df = pd.DataFrame()
df['PostalCode'] = postCodeList
df['Borough'] = boroughList
df['Neighbourhood'] = neighbourhoodList

df = df[df.Borough != 'Not assigned']
df = df.groupby(('PostalCode','Borough')).agg([('Neighbourhood', ', '.join)]).reset_index()

for index, row in df.iterrows():
    neighbourhood = row['Neighbourhood']

    if neighbourhood.item() == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

df_scrapped = pd.DataFrame()
df_scrapped['PostalCode']= df.iloc[:,0].copy()
df_scrapped['Borough']= df.iloc[:,1].copy()
df_scrapped['Neighbourhood']= df.iloc[:,2].copy()

In [67]:
df_scrapped.head(21)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [68]:
df_scrapped.shape

(103, 3)